In [127]:
import matplotlib as mpl
from joblib import Parallel, delayed
import tqdm
from scipy.optimize import curve_fit
import os
import pandas as pd
from collections import OrderedDict
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append('../scales_project/')
from utils import utils
from utils import scale_by_scale_optim
from utils import scale_fitter_no_grid
from utils import evaluate
from utils import simulate_EPR


In [128]:


def individual_locations(x,col='loc'):
    ll = list(OrderedDict.fromkeys(x[col].values))
    return x[col].map(dict(zip(ll,range(len(ll)))))

def fit_scales(data_u, useruuid):

    
    # Stop locations median #
    # ------------------- #
    data_u['lat'] = data_u.groupby('loc')['lat'].transform('median')
    data_u['lon'] = data_u.groupby('loc')['lon'].transform('median')

    #data_u = data_u[data_u['loc']!=data_u['loc'].shift()]
    data_u = data_u.dropna(subset = ['lat','lon'])
    #data_u['loc'] = data_u['loc'].map(dict(zip(set(data_u['loc']),range(len(data_u['loc'])))))

    
    
    stop_coords = data_u.groupby('loc')[['lat','lon']].median().values
    series = np.array(list(map(int, data_u['loc'])))


    # Find scale solution #
    # ------------------- #
    
    my_split = scale_by_scale_optim.ScalesOptim(series, 
                                                stop_coords,
                                                min_dist = 1.2, 
                                                nprocs = 1,
                                                bootstrap = True,
                                                information_criterion = None,
                                                siglvl=0.4, 
                                                verbose = False)  
    final_series, final_scales, likelihoods, criterion_s, final_sizes, final_proba_dist, final_alphas = my_split.find_best_scale()
    
    # Find number of containers per scale #
    # ------------------------------------ #
    containers_per_scale = [len(set(np.array(final_series)[:,i])) for i in range(len(final_series[0]))]
  
    trip_labels = [np.nan]+utils.get_scale_labels(final_series)
    
    data_u['trip_scale'] = trip_labels
    data_u['sizes'] = [list(final_sizes.values())]*len(data_u)
    data_u['scales'] = [list(final_scales.values())]* len(data_u)
    data_u['containers'] = [list(containers_per_scale)]* len(data_u)
    data_u['max_d'] = my_split.max_d
    data_u['final_series'] = final_series

    return data_u

def fit_scale_pandas(dataframe_user, min_days):
    user_name = dataframe_user['useruuid'].iloc[0]
    days = set((dataframe_user.start/(60*60*24)).astype(int))
    if ((dataframe_user.start.max() - dataframe_user.start.min())>=(min_days*60*60*24)) and len(days)>min_days:
        dataframe_user = fit_scales(dataframe_user,user_name)

        #Find home
        i = dataframe_user.groupby('loc')['lat'].transform('size').sort_values(ascending = False).index.values[0]
        home = dataframe_user.loc[i][['lat','lon']].values
        dataframe_user['home'] = [home]*len(dataframe_user)    
        return dataframe_user[['useruuid','loc','start','end','lat','lon','trip_scale','scales','sizes','containers','home','max_d','final_series']]
    else:
        return pd.DataFrame([],columns = ['useruuid','loc','start','end','lat','lon','trip_scale','scales','sizes','containers','home','max_d','final_series'])    
        

In [129]:
MIN_DAYS = 250
LEN_TRAINING = 300
LEN_TEST = 50
N_TRACES = 100
N_JOBS = 20
BINS_DR = np.logspace(1,7,40)
N_USERS = 9000

# CREATE TRAIN AND TEST SSET


In [50]:
buckets = os.listdir('/data/work/user/laura_data/scale_trips_0.5/')[:60]

In [51]:
user_data = pd.concat([pd.read_parquet('/data/work/user/laura_data/scale_trips_0.5/{}/'.format(i)) for i in buckets])
user_data['loc'] = user_data['final_series'].apply(lambda x:x[-1])
user_data = user_data[user_data['loc']!=-1].copy()
user_data = user_data[['useruuid','loc','lat','lon','start','end']].copy()

In [52]:
#MAKE SURE LOCATIONS GO FROM 0 to N
user_data = user_data.sort_values(by = ['useruuid','start']).reset_index()
user_data = user_data[user_data.groupby('useruuid')['loc'].shift()!=user_data['loc']]
user_data['loc'] = (user_data.groupby('useruuid').apply(individual_locations)).values


0it [11:10, ?it/s]


In [53]:
#N months of training
training_data = user_data[user_data['start']<user_data.groupby('useruuid')['start'].transform('min')+(LEN_TRAINING*24*60*60)].copy()
#50 steps of testinguseruuid
testing_data = (user_data[user_data['start']>=user_data.groupby('useruuid')['start'].transform('min')+(LEN_TRAINING*24*60*60)]).groupby('useruuid').head(LEN_TEST)

In [54]:
del user_data

# FIT CONTAINER MODEL TO TRAINING SET

In [130]:
first_time = False

if first_time==True:
    #Find scales
    df_final = Parallel(n_jobs=25)(delayed(fit_scale_pandas)(group, MIN_DAYS) for key, group in tqdm.tqdm(training_data.groupby('useruuid')))
    df_final = pd.concat([i for i in df_final if len(i)>0])
    users_scales = df_final[['useruuid','scales','sizes','containers','home','max_d']].drop_duplicates(subset = ['useruuid'])
    users_trips = df_final[['useruuid','start','end','lat','lon','trip_scale','final_series','loc']]
    users_scales.to_pickle('./modelling/scales_global.pkl')
    users_trips.to_pickle('./modelling/scale_trips_global.pkl')

    
else:
    users_trips = pd.read_pickle('./modelling/scale_trips_global.pkl')
    users_trips = users_trips[['useruuid','start','end','lat','lon','trip_scale','final_series','loc']].copy()
    users_scales = pd.read_pickle('./modelling/scales_global.pkl')


In [56]:
len(users_trips['useruuid'].unique())

15047

In [57]:
len(training_data["useruuid"].unique())

140390

In [58]:
len(users_trips['useruuid'].unique())

15047

# SIMULATIONS

In [125]:
def generate_data_scales(key, 
                         group, 
                         N_traces, 
                         length=50):
    
    """
    Generate traces using the scales model
    key: user_id
    group: output of the scales fit
    N_traces : number of traces
    length: number of displacemnts
    
    """
    
    data = group['final_series'].values
    stop_coords = [tuple(i) for i in group.groupby('loc')[['lat','lon']].median().values]
    simulations = pd.DataFrame(columns=['loc','coords','sim'])
    for n in range(N_traces):
        d = pd.DataFrame(evaluate.scales_generator(data,length,stop_coords),columns=['loc','coords'])
        d['sim'] = n
        simulations = pd.concat([simulations,d])
    simulations['sim'] = simulations['sim'].astype(int)
    simulations.to_pickle("./modelling/scales_synthetic_3/{}".format(key))
    return simulations

def generate_data_epr(key,group, 
                      N_traces, 
                      length=50,
                      rho=0.6,
                      gamma=0.21,
                      alpha=0.55):
    """
    Generate traces using the epr model
    key: user_id
    group: output of the scales fit
    N_traces : number of traces
    length: number of displacemnts
    rho, gamma, alpha: parameters of the model
    
    """
    if os.path.exists("./modelling/epr_synthetic_3/{}".format(key)):
        return 0
    group = group.copy()
    model = simulate_EPR.EPRModel(distance='haversine',p_rho=rho,p_gamma=gamma,p_alpha=alpha)
    stop_coords = [tuple(i) for i in group.groupby('loc')[['lat','lon']].median().sort_index().values]

    simulations = pd.DataFrame(columns=['loc','coords','sim'])
    for n in range(N_traces):
        model.reset()
        model.train(group['loc'].values,list(stop_coords))
        model.run_simulation(length)
        d = pd.DataFrame(model.records,columns=['loc','coords'])
        d['sim'] = n
        simulations = pd.concat([simulations,d])

    pd.DataFrame(simulations).to_pickle("./modelling/epr_synthetic_3/{}".format(key))

    return simulations

def generate_data_m_epr(key, 
                        group, 
                        N_traces, 
                        length=50,
                        rho=0.6,
                        gamma=0.21,
                        alpha=0.55, 
                        memory=30):
    
    """
    Generate traces using the m_epr model
    key: user_id
    group: output of the scales fit
    N_traces : number of traces
    length: number of displacemnts
    rho, gamma, alpha: parameters of the model
    
    """ 
    if os.path.exists("./modelling/m_epr_synthetic_3/{}".format(key)):
        return 0
    group = group.copy()
    model = simulate_EPR.EPRModel_with_memory(distance='haversine',memory=memory,p_rho=rho,p_gamma=gamma,p_alpha=alpha)
    stop_coords = [tuple(i) for i in group.groupby('loc')[['lat','lon']].median().sort_index().values]


    simulations = pd.DataFrame(columns=['loc','coords','sim'])
    for n in range(N_traces):
        model.reset()
        model.train(group['loc'].values,list(stop_coords))
        model.run_simulation(length)
        d = pd.DataFrame(model.records,columns=['loc','coords'])
        d['sim'] = n
        simulations = pd.concat([simulations,d])

    pd.DataFrame(simulations).to_pickle("./modelling/m_epr_synthetic_3/{}".format(key))

    return simulations


def generate_data_recency_epr(key, 
                              group, 
                              N_traces, 
                              length=50,
                              rho=0.6,
                              gamma=0.21,
                              alpha=0.55):
    """
    Generate traces using the m_epr model
    key: user_id
    group: output of the scales fit
    N_traces : number of traces
    length: number of displacemnts
    rho, gamma, alpha: parameters of the model
    
    """ 
    if os.path.exists("./modelling/recency_epr_synthetic_3/{}".format(key)):
        return 0
    group = group.copy()
    model = simulate_EPR.RecencyEPRModel(distance='haversine',p_rho=rho,p_gamma=gamma,p_alpha=alpha)
    
    stop_coords = group.groupby('loc')[['lat','lon']].median().sort_index().apply(lambda x:tuple(x),axis =1).to_dict()

    simulations = pd.DataFrame(columns=['loc','coords','sim'])
    for n in range(N_traces):
        model.reset()
        model.train(group['loc'].values,stop_coords)
        model.run_simulation(length)
        d = pd.DataFrame(model.records,columns=['loc','coords'])
        d['sim'] = n
        simulations = pd.concat([simulations,d])

    pd.DataFrame(simulations).to_pickle("./modelling/recency_epr_synthetic_3/{}".format(key))



    return simulations



# COMPUTE AGGREGATED STATISTICS FOR REAL DATA

In [44]:
def compute_dr(trace_coordinates, 
               bins = np.logspace(2,7,40)):
    """
    Compute probability of displacements
    trace_coordinates: list of lat,lon
    """
    starts = np.array([list(i) for i in trace_coordinates[1:]])
    ends = np.array([list(i) for i in trace_coordinates[:-1]])
    distances = utils.haversine(starts,ends)
    return np.histogram(distances, bins = bins,density = True)[0]


def compute_radius(trace_locations,trace_coordinates):
    """
    Compute radius of gyration.
    trace_locations: list of location labels
    trace_coordinates: list o lat,lon
    """
    home_location = sorted(Counter(trace_locations).items(),key = lambda x:x[1])[-1][0]    
    index = np.where(trace_locations==home_location)[0][0]
    trace_coordinates = np.array([list(i) for i in trace_coordinates])[index:]
    center_of_mass = [np.median(trace_coordinates[:i],axis =0) for i in range(1,len(trace_coordinates)+1)]

    
    rg =  [np.abs(i) for i in utils.haversine(trace_coordinates, center_of_mass)]
    rg = np.cumsum(rg) / np.arange(1,len(rg)+1)

    return rg


def compute_distr_locations(trace_locations, N_loc=100):
    """
    Compute frequency rank.
    trace_locations: list of location labels
    N: top locations to consider
    """
    x = trace_locations
    xx = [i/len(x) for i in sorted(Counter(x).values(), reverse = True)]
    N = N_loc - len(xx)
    s = sum(xx[:N_loc])
    xx = [i/s for i in xx[:N_loc]]+[0]*N
    return xx

def compute_entropies_iteration(trace_locations, 
                                return_median=True):
    """
    Compute entropy.
    trace_locations: list of location labels
    N: top locations to consider
    """
    if len(trace_locations)>=LEN_TEST:
        temp_entropy = evaluate.est_entropy(trace_locations)
        aa = []
        for _ in range(1):
            unc_entropy = evaluate.est_entropy(np.random.choice(trace_locations, len(trace_locations), replace = False))
            aa.append((temp_entropy - unc_entropy))
        if return_median:
            return np.median(aa)
        else:
            return aa
    else:
        if return_median:
            return np.nan
        else:
            return []
    
    

def compute_everything(trace):
    """
    Compute all aggregated statistics
    """
    if len(trace)>1:
        trace_locations = trace['loc'].values
        trace_coordinates = trace['coords'].values
        dr = compute_dr(trace_coordinates, bins = BINS_DR)
        radius = compute_radius(trace_locations,trace_coordinates)
        distr_locations = compute_distr_locations(trace_locations)
        entropy = compute_entropies_iteration(trace_locations)
        return dr,radius,distr_locations,entropy
    else:
         return np.array([]),np.array([]),np.array([]),np.nan

In [61]:
users = testing_data.groupby('useruuid').first()[testing_data.groupby('useruuid').size()==LEN_TEST].index.values
users = list(set(users).intersection(set(users_trips.useruuid)))[:N_USERS]

In [62]:
## COMPUTE PROPERTIES REAL DATA
training_data_filtered = training_data[training_data.useruuid.isin(users)].copy()
testing_data_filtered = testing_data[testing_data.useruuid.isin(users)].copy()

testing_data_filtered['coords'] = testing_data_filtered[['lat','lon']].apply(lambda x:tuple(x),axis = 1)
training_data_filtered['coords'] = training_data_filtered[['lat','lon']].apply(lambda x:tuple(x),axis = 1)

In [105]:
testing_data_filtered.to_pickle("./modelling/test_data_3.pkl")

In [107]:

###COMPUTE PROPERTIES OF TRAINING AND TESTING DATA
properties_training = Parallel(n_jobs=N_JOBS)(delayed(compute_everything)(group) for key, group in 
                                          tqdm.tqdm(training_data_filtered.groupby("useruuid")))
properties_testing = Parallel(n_jobs=N_JOBS)(delayed(compute_everything)(group) for key, group in 
                                         tqdm.tqdm(testing_data_filtered.groupby("useruuid")))


100%|██████████| 9000/9000 [01:36<00:00, 93.59it/s] 


# RUN  SIMULATIONS


In [108]:
#FIND PARAMETERS

#FIND EXPONENT OF THE POWER LAW DISTRIBUTION OF DISPLACEMENTS
e = BINS_DR
test = np.nanmean([i[0] for n,i in enumerate(properties_training) if len(i[0])>0],axis = 0)
a, b = np.polyfit(np.log(e[5:-1]),np.log(test[5:]),1)



#FIND RHO E GAMMA

def count_locations(trace):
    locations = set()
    wait_before_discovery= [1]
    S = 0
    for loc in trace:
        if loc in locations:
            wait_before_discovery[S]+=1
        else:
            locations.update({loc})
            wait_before_discovery.append(1)
            S+=1
    return [1/i for i in wait_before_discovery]
            
test = training_data.groupby('useruuid')['loc'].apply(count_locations)
N = LEN_TEST #time to consiser
p_S = np.nanmean([list(i[:N])+[np.nan]*(N-len(i)) for i in test.values], axis = 0)
(rho, gamma),pcov = curve_fit(lambda S, rho, gamma: rho*S**(-gamma), range(1,N+1),p_S)

In [109]:
print(a,rho,gamma)

-1.5157308434588996 0.9532176965416148 0.17952985578134117


In [110]:
np.save("./modelling/properties_testing.npy",properties_testing, allow_pickle=True)
np.save("./modelling/properties_training.npy",properties_training, allow_pickle=True)

/home/lauale/.conda/envs/laura_test/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [114]:
os.makedirs("./modelling/scales_synthetic_3/")
os.makedirs("./modelling/epr_synthetic_3/")
os.makedirs("./modelling/m_epr_synthetic_3/")
os.makedirs("./modelling/recency_epr_synthetic_3/")



In [262]:
a,rho,gamma = -1.5157308434588996, 0.9532176965416148, 0.17952985578134117
users = os.listdir("./modelling/scales_synthetic_3/")
len(users)

9000

In [269]:
first_time=True
filtered_data = users_trips[users_trips.useruuid.isin(users)].groupby("useruuid")

if first_time==True:

    
    #Simulate
    #traces = Parallel(n_jobs=N_JOBS)(delayed(generate_data_scales)(key,group,N_TRACES) for key, group in tqdm.tqdm(filtered_data))
    #traces2 = Parallel(n_jobs=N_JOBS)(delayed(generate_data_epr)(key,group,N_TRACES,rho=rho,gamma=gamma,alpha=-1-a) for key, group in tqdm.tqdm(filtered_data))
    #traces4 = Parallel(n_jobs=N_JOBS)(delayed(generate_data_m_epr)(key,group, N_TRACES, rho=rho,gamma=gamma,alpha=-1-a) for key, group in tqdm.tqdm(filtered_data))
    traces6 = Parallel(n_jobs=N_JOBS)(delayed(generate_data_recency_epr)(key,group, N_TRACES,rho=rho,gamma=gamma,alpha=-1-a) for key, group in tqdm.tqdm(filtered_data))






  0%|          | 0/52 [00:00<?, ?it/s]


100%|██████████| 52/52 [00:02<00:00, 20.73it/s]


In [116]:
#filtered_data = users_trips[users_trips.useruuid.isin(users)].groupby("useruuid")

#C = set([key for key,group in filtered_data])
A = set(testing_data_filtered.useruuid.unique())
B = set(os.listdir("./modelling/epr_synthetic_3/"))
len(A.difference(B)), len(B.difference(A)), len(C.difference(A)), len(C.difference(B))

(0, 0, 0, 0)

# Aggregated stats for simulations

In [6]:
def read_data(folder_name, N = -1):
    users = os.listdir("./modelling/{}/".format(folder_name))  
    trace = []
    for user in users[:N]:
        d = pd.read_pickle("./modelling/{}/{}".format(folder_name, user))
        d['useruuid'] = user
        trace.append(d)
    trace = pd.concat(trace)
    return trace

def compute_average_for_simulated_data(d):

        
    prop_sim = [compute_everything(group[:LEN_TEST]) for key,group in d.groupby("useruuid")]

    a = np.mean(np.array([i[0] for i in prop_sim]),axis = 0)
    a_2 = np.std(np.array([i[0] for i in prop_sim]),axis = 0)


    b = np.nanmedian(np.array([list(i[1])+[np.nan]*(LEN_TEST-len(i[1])) for i in prop_sim]),axis = 0)
    b_2 = np.nanpercentile(np.array([list(i[1])+[np.nan]*(LEN_TEST-len(i[1])) for i in prop_sim]),25,axis = 0)
    b_3 = np.nanpercentile(np.array([list(i[1])+[np.nan]*(LEN_TEST-len(i[1])) for i in prop_sim]),75,axis = 0)

    c = np.mean(np.array([i[2] for i in prop_sim]),axis = 0)
    c_2 = np.std(np.array([i[2] for i in prop_sim]),axis = 0)

    d = np.array([i[3] for i in prop_sim])
    prop_sim = a,b,c,d
    err_sim = a_2, b_2, b_3, c_2
        

    return prop_sim, err_sim


In [7]:
N_JOBS=5
##SIMULATIONS
props = []
errs = []
for folder in ['epr_synthetic_3','scales_synthetic_3','m_epr_synthetic_3','recency_epr_synthetic_3']:
    d = read_data(folder)
    prop, err =  zip(*Parallel(n_jobs=N_JOBS)(delayed(compute_average_for_simulated_data)(d[d.sim==sim].copy()) for sim in tqdm.tqdm(d.sim.unique())))
    props.append(prop)
    errs.append(err)

100%|██████████| 100/100 [1:45:24<00:00, 63.24s/it]  


In [8]:
##HERE ARE THE PROPERTIES OF THE SIMULATED DATA (FOR FINAL FIGURE)
np.save("./modelling/properties_synthetic_data.npy",np.array(props), allow_pickle=True)
np.save("./modelling/errors_synthetic_data.npy",np.array(errs), allow_pickle=True)

<ipython-input-8-92181e31a172>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.save("./modelling/properties_synthetic_data.npy",np.array(props), allow_pickle=True)
<ipython-input-8-92181e31a172>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.save("./modelling/errors_synthetic_data.npy",np.array(errs), allow_pickle=True)


# Compute Likelihood

In [288]:
###DISTRIBUTION OF DISPLACEMENTS
def mean_dr_distr(group_user,bins = BINS_DR):
    """
    Compute average distribution across simulations
    """
    return np.nanmean([compute_dr(group2['coords'][:50].values,bins = bins) for key2, group2 in group_user.groupby('sim')],axis = 0)


def compute_likelihood_dr(trace_coordinates,
                          hist,
                          edges= np.logspace(3,7,40),
                          len_trace=50):
    '''
    Compute likelihood of the trace given the hist
    
    '''

    if len(trace_coordinates)<2:
        return []
    
    
    ##COMPUTE DISTANCES
    starts = np.array([list(i) for i in trace_coordinates[1:]])
    ends = np.array([list(i) for i in trace_coordinates[:-1]])
    distances = utils.haversine(starts,ends)
   
    #FIND PROBABILITY OF EACH DISTANCE UNDER THE COMPUTED HIST
    digitized = []
    n=0
    for i in np.digitize([k for k in distances if k>edges[0] and k<edges[-1]], edges):
        if  hist[i-1]>0:
            digitized.append(hist[i-1]*(edges[i] - edges[i-1]))
        elif hist[i-1]<=0:
            digitized.append(0.0000000000001 )
        n+=1
    
    #RETURN LIKELIHOOD
    L1 = np.log(digitized)#np.sum(-np.log(digitized))/n*len_trace
    return L1

    
###DISTRIBUTION OF LOCATIONS

def compute_pdf_locs(x,bins):
    """
    Compute distribution of location frequencies.
    """
    counts = list(Counter(x).values())
    relative_counts = [i/sum(counts) for i in counts]
    return np.histogram(relative_counts, bins = bins, density = True)[0]
    
    
def pdf_location(group_user, bins):
    return np.mean([compute_pdf_locs(group2['loc'].values,bins) for key2, group2 in group_user.groupby('sim')],axis = 0)



def compute_likelihood_locations(trace_locs,
                                 hist, 
                                 edges= np.logspace(3,7,40),
                                 len_trace=50):
    if len(trace_locs)<2:
        return []
    
    
    ##COMPUTE DISTANCES
    counts = list(Counter(trace_locs).values())
    relative_counts = [i/sum(counts) for i in counts]

    #FIND PROBABILITY OF EACH DISTANCE UNDER THE COMPUTED HIST
    digitized = []
    n=0
    for i in np.digitize([k for k in relative_counts if k>edges[0] and k<edges[-1]], edges):
        if  hist[i-1]>0:
            digitized.append(hist[i-1]*(edges[i] - edges[i-1])) #probability to be in that bin
        elif hist[i-1]<=0:
            digitized.append(0.0000000000001 )
        n+=1
    
    #RETURN LIKELIHOOD
    L1 = np.log(digitized)#np.sum(-np.log(digitized))/n*len_trace #here I just make sure that we have 50 displacements for each user
    return np.sum(L1)



###DISTRIBUTION OF ENTROPY DIFFERENCE

def compute_entropy_hist(trace_locations,bins=np.linspace(-2,2,40)):
    aa = compute_entropies_iteration(trace_locations, return_median=False)
    hist,edges = np.histogram(aa,bins=bins,density=True)
    return hist

def pdf_entropy(group_user, bins=np.linspace(-2,2,40)):
    return np.nanmean([compute_entropy_hist(group2['loc'].values,bins) for key2, group2 in group_user.groupby('sim')],axis = 0)



def compute_likelihood_entropy(trace_locs,
                             hist, 
                             edges= np.linspace(-2,2,40),
                             len_trace=50):
    if len(trace_locs)<2:
        return np.nan
    
    
    ##COMPUTE ENTROPIES
    entropies = compute_entropies_iteration(trace_locs, return_median=False)

    #FIND PROBABILITY OF EACH DISTANCE UNDER THE COMPUTED HIST
    digitized = []
    n=0
    for i in np.digitize([k for k in entropies if k>edges[0] and k<edges[-1]], edges):
        if  hist[i-1]>0:
            digitized.append(hist[i-1]*(edges[i] - edges[i-1]))
        elif hist[i-1]<=0:
            digitized.append(0.0000000000001 )
        n+=1
    
    #RETURN LIKELIHOOD
    L1 = np.log(digitized)#np.sum(-np.log(digitized))/n*len_trace
    return L1


In [181]:
testing_data_filtered = pd.read_pickle("./modelling/test_data_3.pkl")

In [270]:
def compute_dr_akaike(name,testing_data, bins = np.logspace(3,7,40)):
    d = read_data(name,N=-1)
    pdfs =  Parallel(n_jobs=-1,backend='loky')(delayed(mean_dr_distr)(group.copy(),bins) for key,group in tqdm.tqdm(d.groupby("useruuid"),position=0))
    Ls = Parallel(n_jobs=-1,backend='loky')(delayed(compute_likelihood_dr)(testing_data[testing_data.useruuid==(key)]['coords'].values, pdf,bins) for (key,group),pdf in tqdm.tqdm(zip(d.groupby("useruuid"),pdfs),position=0))
    return Ls


def compute_location_akaike(name, testing_data, bins = np.logspace(-6,1,40)):
    d = read_data(name,N=-1)
    pdfs =  Parallel(n_jobs=-1,backend='loky')(delayed(pdf_location)(group.copy(), bins) for key,group in tqdm.tqdm((d.groupby("useruuid")),position=0))
    Ls = Parallel(n_jobs=-1,backend='loky')(delayed(compute_likelihood_locations)(testing_data[testing_data.useruuid==(key)]['loc'].values, pdf,bins) for (key,group),pdf in tqdm.tqdm(zip(d.groupby("useruuid"),pdfs),position=0))

    return Ls

def compute_entropy_akaike(name,testing_data,  bins = np.linspace(-2,2,40)):
    d = read_data(name,N=-1)
    pdfs =  Parallel(n_jobs=-1,backend='loky')(delayed(pdf_entropy)(group.copy(), bins) for key,group in tqdm.tqdm(((d.groupby("useruuid"))),position=0))
    Ls = Parallel(n_jobs=-1,backend='loky')(delayed(compute_likelihood_entropy)(testing_data[testing_data.useruuid==(key)]['loc'].values, pdf, bins) for (key,group),pdf in tqdm.tqdm(zip(d.groupby("useruuid"),pdfs),position=0))
    return Ls


def loglikelihood_ratio(loglikelihoods1, loglikelihoods2,
        nested=False, normalized_ratio=False):
    """
    Calculates a loglikelihood ratio and the p-value for testing which of two
    probability distributions is more likely to have created a set of
    observations.

    Parameters
    ----------
    loglikelihoods1 : list or array
        The logarithms of the likelihoods of each observation, calculated from
        a particular probability distribution.
    loglikelihoods2 : list or array
        The logarithms of the likelihoods of each observation, calculated from
        a particular probability distribution.
    nested : bool, optional
        Whether one of the two probability distributions that generated the
        likelihoods is a nested version of the other. False by default.
    normalized_ratio : bool, optional
        Whether to return the loglikelihood ratio, R, or the normalized
        ratio R/sqrt(n*variance)

    Returns
    -------
    R : float
        The loglikelihood ratio of the two sets of likelihoods. If positive,
        the first set of likelihoods is more likely (and so the probability
        distribution that produced them is a better fit to the data). If
        negative, the reverse is true.
    p : float
        The significance of the sign of R. If below a critical value
        (typically .05) the sign of R is taken to be significant. If above the
        critical value the sign of R is taken to be due to statistical
        fluctuations.
    """
    from numpy import sqrt
    from scipy.special import erfc

    n = float(len(loglikelihoods1))

    if n==0:
        R = 0
        p = 1
        return R, p
    from numpy import asarray
    loglikelihoods1 = asarray(loglikelihoods1)
    loglikelihoods2 = asarray(loglikelihoods2)

    #Clean for extreme values, if any
    from numpy import inf, log
    from sys import float_info
    min_val = log(10**float_info.min_10_exp)
    loglikelihoods1[loglikelihoods1==-inf] = min_val
    loglikelihoods2[loglikelihoods2==-inf] = min_val

    R = sum(loglikelihoods1-loglikelihoods2)

    from numpy import mean
    mean_diff = mean(loglikelihoods1)-mean(loglikelihoods2)
    variance = sum(
            ( (loglikelihoods1-loglikelihoods2) - mean_diff)**2
            )/n

    if nested:
        from scipy.stats import chi2
        p = 1 - chi2.cdf(abs(2*R), 1)
    else:
        p = erfc( abs(R) / sqrt(2*n*variance))

    if normalized_ratio:
        R = R/sqrt(n*variance)

    return R, p

In [272]:

ls_dr = [compute_dr_akaike(i, 
                                 testing_data_filtered, 
                                 bins = np.logspace(1,7,40)) 
                                 for i in 
                       ['scales_synthetic_3','epr_synthetic_3','m_epr_synthetic_3','recency_epr_synthetic_3']]

0it [57:35, ?it/s]
383it [31:39,  4.96s/it]
100%|██████████| 8999/8999 [00:52<00:00, 170.66it/s]
8999it [03:34, 41.95it/s]
100%|██████████| 8999/8999 [00:52<00:00, 172.43it/s]
8999it [03:33, 42.13it/s]
100%|██████████| 8999/8999 [00:42<00:00, 213.19it/s]
8999it [03:31, 42.46it/s]
100%|██████████| 8999/8999 [00:37<00:00, 239.85it/s]
8999it [03:31, 42.54it/s]


In [273]:
ls_entr = [compute_entropy_akaike(i, 
                                 testing_data_filtered, 
                                 bins = np.linspace(-2,2,40)) 
                                 for i in 
                       ['scales_synthetic_3','epr_synthetic_3','m_epr_synthetic_3','recency_epr_synthetic_3']]

100%|██████████| 8999/8999 [01:14<00:00, 120.08it/s]
8999it [03:35, 41.82it/s]
100%|██████████| 8999/8999 [01:05<00:00, 137.02it/s]
8999it [03:36, 41.49it/s]
100%|██████████| 8999/8999 [01:03<00:00, 141.05it/s]
8999it [03:37, 41.36it/s]
100%|██████████| 8999/8999 [00:54<00:00, 164.24it/s]
8999it [03:38, 41.15it/s]


In [275]:
ls_location = [compute_location_akaike(i, 
                                 testing_data_filtered, 
                                 bins = np.logspace(-6,2,40)) 
                                 for i in 
                       ['scales_synthetic_3','epr_synthetic_3','m_epr_synthetic_3','recency_epr_synthetic_3']]

100%|██████████| 8999/8999 [00:34<00:00, 259.47it/s]
8999it [03:30, 42.84it/s]
100%|██████████| 8999/8999 [00:38<00:00, 235.41it/s]
8999it [03:30, 42.66it/s]
100%|██████████| 8999/8999 [00:28<00:00, 310.78it/s]
8999it [03:29, 42.86it/s]
100%|██████████| 8999/8999 [00:28<00:00, 314.06it/s]
8999it [03:32, 42.32it/s]


In [293]:
def significance(p):
    if p<0.001:
        return "***"
    elif p<0.01:
        return "**"
    elif p<0.05:
        return "*"
    else:
        return ""
names = ['scales','epr','m_epr','recency_epr']
from collections import defaultdict
ratios = defaultdict(dict)
names_distr = ['$P(\Delta r)$', "$P(f_l)$","$P(S_{unc} - S_{temp})$"]

for n1, L in enumerate([ls_dr,ls_location,ls_entr]):
    if n1!=1:
        lognorms = [i for i in np.concatenate(L[0]) if not np.isnan(i)]
    else:
        lognorms = L[0]
    for n2,_ in enumerate(L):
        if n1!=1:
            other = [i for i in np.concatenate(L[n2]) if not np.isnan(i)]
        else:
            other = L[n2]
        R,p = loglikelihood_ratio(lognorms,other)
        print(R,p)
        ratios[names_distr[n1]][names[n2]] = str(int(R))+significance(p)
        


0.0 nan
81659.73839917098 0.0
367763.97806551756 0.0
253090.63495127918 0.0
0.0 nan
93296.63554165776 0.0
87617.89005702062 0.0
497676.6884752944 0.0
0.0 nan
44490.701259172965 3.0325751576552672e-258
47140.92986665705 4.1700139467009516e-281
108554.08105607357 0.0
